In [ ]:
%load_ext autoreload
%autoreload 2

from rs_datasets import MovieLens
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from pyspark.sql import functions as sf, types as st
from pyspark.sql.types import IntegerType
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
from sklearn.preprocessing import OneHotEncoder
from replay.utils.spark_utils import convert2spark
from replay.experimental.scenarios.movielens_wrapper.replay_offline import OBPOfflinePolicyLearner
from replay.experimental.scenarios.movielens_wrapper.utils import get_est_rewards_by_reg, bandit_subset

from replay.models import UCB, RandomRec, LinUCB
from tqdm import tqdm

In [ ]:
from obp.ope import (
    OffPolicyEvaluation,
    DirectMethod,
    InverseProbabilityWeighting,
    SelfNormalizedInverseProbabilityWeighting,
    SelfNormalizedDoublyRobust,
)

from replay.experimental.scenarios.offpolicy.modified_ips import Exp_Smooth_IPS_Min, Exp_Smooth_IPS_Max

In [ ]:
import warnings
warnings.filterwarnings("ignore")

### Data preparation

In [ ]:
data = MovieLens("1m")

In [ ]:
N_ITEMS = 1500
N_USERS = 600
name_dir = 'out_exp' #name of the directory to save the results 

In [ ]:
data.ratings['timestamp'] = pd.to_datetime(data.ratings['timestamp'], unit='s')

In [ ]:
data.ratings

In [ ]:
logs = data.ratings.copy()
logs['cnt'] = 1
logs = logs[['item_id', 'cnt']].groupby(by=["item_id"]).sum()
logs = logs.sort_values(by=['cnt'], ascending=False).reset_index()
popular_items = logs.iloc[:N_ITEMS]['item_id'].tolist()

In [ ]:
data.ratings = data.ratings[data.ratings['item_id'].isin(popular_items)]
print('размер датасета логов после выброса непопулярных айтемов:', data.ratings.shape)

In [ ]:

logs = data.ratings.copy()
logs['cnt'] = 1
logs = logs[['user_id', 'cnt']].groupby(by=["user_id"]).sum()
logs = logs.sort_values(by=['cnt'], ascending=False).reset_index()
popular_users = logs.iloc[:N_USERS]['user_id'].tolist()

In [ ]:
data.ratings = data.ratings[data.ratings['user_id'].isin(popular_users)]
print('размер датасета логов после выброса непопулярных юзеров:', data.ratings.shape)

In [ ]:

items = set(data.ratings['item_id'].tolist())
users = set(data.ratings['user_id'].tolist())

In [ ]:
data.items = data.items[data.items['item_id'].isin(items)]
data.users = data.users[data.users['user_id'].isin(users)]
data.users.shape, data.items.shape

In [ ]:
data.ratings

In [ ]:
def features_svd(log, user_features, rank, item_id='item_id', user_id='user_id'):

    interaction_matrix = csr_matrix(
                                        (log['relevance'],
                                        (log[user_id], log[item_id]), #subtract 1 to start with [0,1,2,...]
                                    ),
                                    shape=(N_USERS, N_ITEMS),
                                    dtype=float)

    u, singular_values, vh = svds(
        interaction_matrix,
        k=rank,
    )
 
    svd_feat = pd.DataFrame(np.c_[np.arange(N_USERS),u], columns=[user_id]+[f'svd_feat_{i}' for i in range(rank)])

    return pd.merge(user_features, svd_feat, on=user_id)

In [ ]:
def frac_genres(log, user_features, item_features, item_id='item_id', user_id='user_id'):
    merged = pd.merge(log, item_features, on=item_id)

    # Суммируем жанры по пользователям
    genre_counts = merged.groupby(user_id)[item_features.columns[1:]].sum()

    # Считаем общее количество фильмов для каждого пользователя
    total_movies = merged.groupby(user_id)[item_id].count()

    # Вычисляем среднее количество фильмов для каждого жанра
    average_genre_counts = genre_counts.div(total_movies, axis=0)

    merged_user_features = pd.merge(user_features, average_genre_counts, on=user_id)
    merged_user_features = merged_user_features.rename(columns = {name: name+'user' for name in item_features.drop(columns=[item_id]).columns.values})
    return merged_user_features


In [ ]:
def preprocess(data, item_features, user_features, item_id='item_id', user_id='user_id'):
        # genres = (item_features.select("item_idx", sf.split("genres", "\|").alias("genres")))
        # genres_list = (genres.select(sf.explode("genres").alias("genre")).distinct().filter('genre <> "(no genres listed)"').toPandas()["genre"].tolist())
        # item_features = genres
        # for genre in genres_list:
        #     item_features = item_features.withColumn(genre, sf.array_contains(sf.col("genres"), genre).astype(IntegerType()))

        # item_features = item_features.drop("genres").cache()
        item_features = item_features.drop(['title'], axis = 1)
        item_features['genres'] = item_features['genres'].str.split('|')

        genres_list = pd.Series([genre for sublist in item_features['genres'] for genre in sublist]).unique()
        genres_list = [genre for genre in genres_list if genre != "(no genres listed)"]
        for genre in genres_list:
            item_features[genre] = item_features['genres'].apply(lambda x: int(genre in x))

        item_features = item_features.drop(columns=['genres'])
        item_features[item_id] = item_features[item_id].astype('int64')

        user_features = user_features.drop(columns=['zip_code'])
        bins = [0, 20, 30, 40, 50, 60, np.inf]
        names = ['<20', '20-29', '30-39','40-49', '51-60', '60+']

        user_features['agegroup'] = pd.cut(user_features['age'], bins, labels=names)
        user_features = user_features.drop(["age"], axis = 1)

        columnsToEncode = ["agegroup","gender","occupation"]

        myEncoder = OneHotEncoder(sparse=False, handle_unknown='ignore') #for old versions use sparse
        myEncoder.fit(user_features[columnsToEncode])

        user_features = pd.concat([user_features.drop(columnsToEncode, 1),
                           pd.DataFrame(myEncoder.transform(user_features[columnsToEncode]), 
                                        columns = myEncoder.get_feature_names_out(columnsToEncode))], axis=1).reindex()

        user_features[user_id] = user_features[user_id].astype('int64')
        user_features = frac_genres(data, user_features, item_features, item_id, user_id)

        user_features =  features_svd(data, user_features, 16, item_id, user_id)
        return item_features, user_features


In [ ]:
def encode_items(data, features, item_id='item_id'):
    """Encode items to consecutive ids."""

    encoder = LabelEncoder()
    data[item_id] = encoder.fit_transform(data[item_id])
    features[item_id] = encoder.transform(features[item_id])
    return data, features


def encode_users(data, features, user_id='user_id'):
    """Encode items to consecutive ids."""

    encoder = LabelEncoder()
    data[user_id] = encoder.fit_transform(data[user_id])
    features[user_id] = encoder.transform(features[user_id])

    return data, features

In [ ]:
def split(data, item_features, user_features, q, item_id='item_id', user_id='user_id'):
    data['relevance'] = data['rating'].apply(lambda x: int(x>=3))
    data = data.drop(['rating'], axis = 1)

    test_timepoint = data['timestamp'].quantile(
          q=q, interpolation='nearest'
    )
    test_data_ = data.query('timestamp >= @test_timepoint')

    train_data_ = data.query(
        'timestamp < @test_timepoint')
    #drop cold items

    test_data_ = test_data_[test_data_[item_id].isin(np.unique(train_data_[item_id].values))]
    full_data  = data[data[item_id].isin(np.unique(train_data_[item_id].values))]
    item_features = item_features[item_features[item_id].isin(np.unique(train_data_[item_id].values))]
    #reindex items

    encoder = LabelEncoder()
    full_data[item_id] = encoder.fit_transform(full_data[item_id])
    test_data_[item_id] = encoder.transform(test_data_[item_id])
    train_data_[item_id] = encoder.transform(train_data_[item_id])
    item_features[item_id] = encoder.transform(item_features[item_id])

    N_ITEMS = item_features.shape[0]

    train_data_ = train_data_.sort_values('timestamp')
    test_data_ = test_data_.sort_values('timestamp')
    full_data = full_data.sort_values('timestamp')

    user_features=user_features.reset_index(drop=True).sort_values(by = [user_id])
    item_features=item_features.reset_index(drop=True).sort_values(by = [item_id])

    return train_data_, test_data_, item_features, user_features, full_data


In [ ]:
log, item_features = encode_items(data.ratings, data.items, item_id='item_id')
log, user_features = encode_users(log, data.users, user_id='user_id')

In [ ]:
train, test, item_features, user_features, post_log = split(log, item_features, user_features, q=0.7, item_id='item_id', user_id='user_id')

In [ ]:
item_features, user_features = preprocess(post_log, item_features, user_features)

In [ ]:
train.rename(columns={"user_id": "user_idx", "item_id": "item_idx"}, inplace=True)
test.rename(columns={"user_id": "user_idx", "item_id": "item_idx"}, inplace=True)
post_log.rename(columns={"user_id": "user_idx", "item_id": "item_idx"}, inplace=True)
item_features.rename(columns={"user_id": "user_idx", "item_id": "item_idx"}, inplace=True)
user_features.rename(columns={"user_id": "user_idx", "item_id": "item_idx"}, inplace=True)

### Data in OBD format

In [ ]:
bandit_feedback_train = dict(
    log= convert2spark(train),
    item_features= convert2spark(item_features),
    user_features= convert2spark(user_features),
    n_rounds= train.shape[0],
    n_actions= item_features.shape[0],
    action = train['item_idx'].values,
    position= np.zeros(train.shape[0]).astype(np.int32),
    reward= train['relevance'].values,
    context= train[['user_idx']].merge(user_features, on='user_idx', how='left').drop(columns=['user_idx']).to_numpy(),
    action_context= item_features.sort_values(by = ['item_idx']).drop(columns=['item_idx']).to_numpy(),
    pscore = np.ones(train.shape[0]) # to write something that's unnecessary for get_est_rewards_by_reg 
)

bandit_feedback_test = dict(
    log=convert2spark(test),
    item_features= convert2spark(item_features),
    user_features= convert2spark(user_features),
    n_rounds= test.shape[0],
    n_actions= item_features.shape[0],
    action = test['item_idx'].values,
    position= np.zeros(test.shape[0]).astype(np.int32),
    reward= test['relevance'].values,
    context= test[['user_idx']].merge(user_features, on='user_idx', how='left').drop(columns=['user_idx']).to_numpy(),
    action_context= item_features.sort_values(by = ['item_idx']).drop(columns=['item_idx']).to_numpy(),
    pscore = np.ones(test.shape[0])  # to write something that's unnecessary for get_est_rewards_by_reg
)


### Rewards estimating

In [ ]:
estimated_rewards_by_reg_model = get_est_rewards_by_reg(bandit_feedback_train['n_actions'],
                                                        1,
                                                        bandit_feedback_train,
                                                        bandit_feedback_test)

### Training bandits models

In [ ]:
model_1 = UCB(exploration_coef = 0.01, sample = True, seed = 123)
learner_1 = OBPOfflinePolicyLearner(n_actions=bandit_feedback_train['n_actions'],
                                    replay_model=model_1)

model_2 = LinUCB(eps = -10, alpha = 10, is_hybrid=False)
learner_2 = OBPOfflinePolicyLearner(n_actions=bandit_feedback_train['n_actions'],
                                    replay_model=model_2)

model_3 = RandomRec(distribution = 'relevance', alpha = 154.0, seed=42)
learner_3 = OBPOfflinePolicyLearner(n_actions=bandit_feedback_train['n_actions'],
                                    replay_model=model_3)


In [ ]:
def get_dist(learner):
    all_action_dist = np.zeros((bandit_feedback_test["n_rounds"], bandit_feedback_test["n_actions"], 1))
    if isinstance(learner.replay_model, (LinUCB)):
        log_distinct = bandit_feedback_test['log'].toPandas().drop_duplicates(subset=["user_idx"], keep='first')
        users_all = bandit_feedback_test['log'].toPandas()['user_idx'].tolist()
        batch_size = 10
        num_batchs = log_distinct.shape[0] // batch_size
        for batch_idx in tqdm(range(num_batchs+1)):
            j = min((batch_idx+1)*batch_size, log_distinct.shape[0])
            if j == batch_idx*batch_size:
                break
            log_subset = log_distinct.iloc[batch_idx*batch_size: j]
            n_rounds = log_subset.shape[0]
            
            action_dist = learner.predict(n_rounds, convert2spark(log_subset).select('user_idx'))

            users_distinct = log_subset['user_idx'].tolist()

            user2ind = {}
            for i in range(n_rounds):
                user2ind[users_distinct[i]] = i

            for i in range(bandit_feedback_test["n_rounds"]):
                if users_all[i] in users_distinct:
                    all_action_dist[i] = action_dist[user2ind[users_all[i]]]

    else:
        batch_size = 300
        num_batchs = bandit_feedback_test["n_rounds"] // batch_size
        for batch_idx in tqdm(range(num_batchs+1)):
            j = min((batch_idx+1)*batch_size, bandit_feedback_test["n_rounds"])
            if j == batch_idx*batch_size:
                break
            bandit_feedback_subset = bandit_subset([batch_idx*batch_size, j], bandit_feedback_test) #The first parameter is a slice of subset [a, b]
            action_dist = learner.predict(bandit_feedback_subset["n_rounds"], bandit_feedback_subset["log"].select('user_idx'))
            all_action_dist[batch_idx*batch_size:j] = action_dist
    return all_action_dist

In [ ]:
learner_1.fit(bandit_feedback_train)
all_action_dist_1 = get_dist(learner_1)

In [ ]:
learner_2.fit(bandit_feedback_train)
all_action_dist_2 = get_dist(learner_2)

In [ ]:
learner_3.fit(bandit_feedback_train)
all_action_dist_3 = get_dist(learner_3)

### Compute classic metrics

In [ ]:
learner_1.predict_and_evaluate_new(bandit_feedback_test, K=10)

In [ ]:
learner_2.predict_and_evaluate_new(bandit_feedback_test, K=10)

In [ ]:
learner_3.predict_and_evaluate_new(bandit_feedback_test, K=10)

In [ ]:
np.save(f'{name_dir}/UCB_policy', all_action_dist_1)
np.save(f'{name_dir}/Lin_UCB_policy', all_action_dist_2)
np.save(f'{name_dir}/Random_policy', all_action_dist_3)

### Train Boosting

In [ ]:
num_polices = 25

In [ ]:
context_train = post_log[['user_idx']].merge(user_features, on='user_idx', how='left').drop(columns=['user_idx']).to_numpy()
action_train =  post_log['item_idx'].values

In [ ]:
import lightgbm as lgb
size_subset = int(0.8*context_train.shape[0])
params = {
    'objective': 'multiclass',
    'num_class': N_ITEMS,
    'metric': 'multi_logloss',
    'verbose': 1
}
behavior_policies = []
cnt_skip = 0
for i in range(num_polices):
    print(i,"start")
    bootstrap_idx = np.random.choice(context_train.shape[0], size=size_subset)
    if np.unique(action_train[bootstrap_idx]).shape[0] != N_ITEMS:
        cnt_skip += 1
        continue
    train_data = lgb.Dataset(context_train[bootstrap_idx], label=action_train[bootstrap_idx])
    test_data = lgb.Dataset(bandit_feedback_test['context'][:10000], bandit_feedback_test['action'][:10000])
    model = lgb.train(params,
                  train_data,
                  100,
                  early_stopping_rounds=10,
                  valid_sets=[test_data])
    probs = model.predict(bandit_feedback_test['context'], num_iteration=model.best_iteration)
    behavior_policies.append(probs[np.arange(bandit_feedback_test['action'].size), bandit_feedback_test['action']])
behavior_policies_np = np.array(behavior_policies)
np.save(f'{name_dir}/behavior_policies_lightgbm_25', behavior_policies_np)
print('cnt skip:', cnt_skip) 

### Train LogReg with KL regularization

In [ ]:
import torch
def CrossEntropyLoss_oh(pred, labels, kl_dist, alpha):
    log_loss = torch.nn.functional.cross_entropy(pred,labels)
    kl_reg = torch.mean(torch.sum((kl_dist * torch.log(kl_dist/torch.nn.functional.softmax(pred, dim=1))), dim=1))
    return log_loss + alpha * kl_reg

class my_Model(torch.nn.Module):
    def __init__(self,cnt_feat, num_classes):
        super().__init__()
        self.lin = torch.nn.Sequential(
            torch.nn.Linear(cnt_feat, num_classes, bias=True)           
        )
        
    def forward(self, x):
        return self.lin(x)

class Behavior_Clone_Model:
    def __init__(self, cnt_feat, num_classes, lr, n_epochs, batch_size, alpha):
        self.n_epochs = n_epochs  # epochs
        self.lr = lr  # learning rate
        self.batch_size = batch_size #batch_size
        self.alpha = alpha #param of regul
        # model
        self.model = my_Model(cnt_feat, num_classes)
        self.num_classes = num_classes

    def fit(self, x_train, Y_train, x_test, Y_test):
        # training graph and optimization
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr)
        # loss and accuracy storage
        cnt_batch_per_epoch = Y_train.size//self.batch_size
        self.model.train()
        for epoch in range(self.n_epochs*cnt_batch_per_epoch + 1):
            # randomic batch definition
            rbatch = np.random.choice(Y_train.size, size=self.batch_size)
            # variables initialization
            X = torch.autograd.Variable(torch.FloatTensor(x_train[rbatch]))
            Y = torch.LongTensor(Y_train[rbatch].astype(np.int))
            dist_batch = torch.FloatTensor(np.ones((self.batch_size, self.num_classes))* (1/self.num_classes))
            # training, metrics and storage
            optimizer.zero_grad()
            pred = self.model(X)
            L = CrossEntropyLoss_oh(pred, Y, dist_batch, self.alpha)
            L.backward()
            optimizer.step()

            if epoch%1000 == 0:
                self.model.eval()
                with torch.no_grad():
                    X_ = torch.autograd.Variable(torch.FloatTensor(x_test))
                    pred_test = self.model(X_)
                    Y_ = torch.LongTensor(Y_test)
                    dist_test = torch.FloatTensor(np.ones((Y_.shape[0], self.num_classes))* (1/self.num_classes))
                    loss_test = CrossEntropyLoss_oh(pred_test, Y_, dist_test, self.alpha)
                    print('epoch', epoch, 'ce_loss_test: ', loss_test.detach())
                print('epoch: {0:04d} | loss: {1:.3f}'.format(epoch, L))
                self.model.train()
    
    def predict_proba(self, x_test, Y_test):
        self.model.eval()
        with torch.no_grad():
            X_ = torch.autograd.Variable(torch.FloatTensor(x_test))
            probs = torch.nn.functional.softmax(self.model(X_), dim=1).detach().numpy()
        print('acc: ',(np.argmax(probs, axis = 1) == Y_test).mean())
        return probs[np.arange(Y_test.size), Y_test]

In [ ]:
size_subset = int(0.8*context_train.shape[0])
cnt_skip = 0
for alpha in [0, 10, 70]:
    behavior_policies = []
    print(alpha, 'start')
    for i in range(num_polices):
        print(i,"start")
        bootstrap_idx = np.random.choice(context_train.shape[0], size=size_subset)
        if np.unique(action_train[bootstrap_idx]).shape[0] != N_ITEMS:
            cnt_skip += 1
            continue
        b_model = Behavior_Clone_Model(context_train.shape[1], bandit_feedback_train['n_actions'], 1e-4, 20, 64, alpha)
        b_model.fit(context_train[bootstrap_idx], action_train[bootstrap_idx], bandit_feedback_test['context'][:10000], bandit_feedback_test['action'][:10000])
        behavior_policies.append(b_model.predict_proba(bandit_feedback_test['context'], bandit_feedback_test['action']))
    behavior_policies_np = np.array(behavior_policies)
    np.save(f'{name_dir}/behavior_policies_log_reg_25_alpha={alpha}', behavior_policies_np)
print('cnt_skip', cnt_skip)

### Compute viewed items for users

In [ ]:
viewed_items = bandit_feedback_train["log"].toPandas().groupby("user_idx", sort=True)["item_idx"].apply(list)
viewed_items_all = { i: viewed_items[i] if i in viewed_items.index else [] for i in range(N_USERS)}
viewed_items_for_test = [viewed_items_all[item_idx] for item_idx in bandit_feedback_test['log'].toPandas()['user_idx'].values]
viewed_items_for_test = np.array(viewed_items_for_test)
viewed_items_for_test

### Define Offpolicy methods 

In [ ]:
def ipw(pi_e, pi_0, r, r_hat):
    weights = pi_e/pi_0
    return weights * r

def snipw(pi_e, pi_0, r, r_hat):
    weights = pi_e/pi_0
    weights = weights/weights.mean()
    return weights * r
    
def sndr(pi_e, pi_0, r, r_hat, dm):
    weights = pi_e/pi_0
    weights = weights/weights.mean()
    return dm + weights*(r-r_hat)

def dm(dm):
    return dm

### Load models distributions 

In [ ]:
all_action_dist_1 = np.load(f"{name_dir}/UCB_policy.npy")
all_action_dist_2 = np.load(f"{name_dir}/Lin_UCB_policy.npy")
all_action_dist_3 = np.load(f"{name_dir}/Random_policy.npy")

### Compute r_hat

In [ ]:
r_hat = estimated_rewards_by_reg_model[np.arange(bandit_feedback_test['action'].shape[0]), bandit_feedback_test['action'], 0]

### Filter viewed items 

In [ ]:
all_action_dist_1_new = all_action_dist_1.copy()
for i in range(all_action_dist_1.shape[0]):
    all_action_dist_1_new[i][viewed_items_for_test[i]] = 0
    all_action_dist_1_new[i]/= all_action_dist_1_new[i].sum()

In [ ]:
all_action_dist_2_new = all_action_dist_2.copy()
for i in range(all_action_dist_2.shape[0]):
    all_action_dist_2_new[i][viewed_items_for_test[i]] = 0
    all_action_dist_2_new[i]/= all_action_dist_2_new[i].sum()

In [ ]:
all_action_dist_3_new = all_action_dist_3.copy()
for i in range(all_action_dist_3.shape[0]):
    all_action_dist_3_new[i][viewed_items_for_test[i]] = 0
    all_action_dist_3_new[i]/= all_action_dist_3_new[i].sum()

### Top 10

In [ ]:
all_action_dist_1_top_10 = all_action_dist_1_new.copy()
idx_1 = np.argsort(-all_action_dist_1_top_10, axis = 1)
for i in range(all_action_dist_1_top_10.shape[0]):
    all_action_dist_1_top_10[i][idx_1[i][10:]] = 0
    all_action_dist_1_top_10[i] /= all_action_dist_1_top_10[i].sum()

In [ ]:
all_action_dist_2_top_10 = all_action_dist_2_new.copy()
idx_2 = np.argsort(-all_action_dist_2_top_10, axis = 1)
for i in range(all_action_dist_2_top_10.shape[0]):
    all_action_dist_2_top_10[i][idx_2[i][10:]] = 0
    all_action_dist_2_top_10[i] /= all_action_dist_2_top_10[i].sum()

In [ ]:
all_action_dist_3_top_10 = all_action_dist_3_new.copy()
idx_3 = np.argsort(-all_action_dist_3_top_10, axis = 1)
for i in range(all_action_dist_3_top_10.shape[0]):
    all_action_dist_3_top_10[i][idx_3[i][10:]] = 0
    all_action_dist_3_top_10[i] /= all_action_dist_3_top_10[i].sum()

### Compute dm

In [ ]:
DM_method_1 = (estimated_rewards_by_reg_model * all_action_dist_1).sum(axis = 1).squeeze()
DM_method_2 = (estimated_rewards_by_reg_model * all_action_dist_2).sum(axis = 1).squeeze()
DM_method_3 = (estimated_rewards_by_reg_model * all_action_dist_3).sum(axis = 1).squeeze()

In [ ]:
DM_method_1_new = (estimated_rewards_by_reg_model * all_action_dist_1_new).sum(axis = 1).squeeze()
DM_method_2_new = (estimated_rewards_by_reg_model * all_action_dist_2_new).sum(axis = 1).squeeze()
DM_method_3_new = (estimated_rewards_by_reg_model * all_action_dist_3_new).sum(axis = 1).squeeze()

In [ ]:
DM_method_1_top_10 = (estimated_rewards_by_reg_model * all_action_dist_1_top_10).sum(axis = 1).squeeze()
DM_method_2_top_10 = (estimated_rewards_by_reg_model * all_action_dist_2_top_10).sum(axis = 1).squeeze()
DM_method_3_top_10 = (estimated_rewards_by_reg_model * all_action_dist_3_top_10).sum(axis = 1).squeeze()

### Compute TV distance

In [ ]:
new_log = post_log.copy()
# new_log = new_log.iloc[bootstrap_idx]
new_test_log = bandit_feedback_test['log'].toPandas().copy()
frequency = new_log.groupby(['user_idx', 'item_idx']).size().reset_index(name='frequency')

# Шаг 2: Считаем сумму всех частот для каждого user
frequency['total_frequency'] = frequency.groupby('user_idx')['frequency'].transform('sum')

# Шаг 3: Нормируем частоты
frequency['normalized_frequency'] = frequency['frequency'] / frequency['total_frequency']
frac_dist_test = [frequency[(frequency['user_idx'] == new_test_log.iloc[i]['user_idx'])&(frequency['item_idx'] == new_test_log.iloc[i]['item_idx'])]['normalized_frequency'].values[0] for i in range(new_test_log.shape[0])]
frac_dist_test = np.array(frac_dist_test)

In [ ]:
random_dist_test = np.ones(bandit_feedback_test['action'].shape[0])/N_ITEMS

In [ ]:
new_log =post_log.copy()
map_pop_dist = new_log['item_idx'].value_counts(normalize=True)
pop_dist = map_pop_dist.loc[new_log['item_idx'].values].values
pop_dist_test = map_pop_dist.loc[bandit_feedback_test['action']].values

In [ ]:
behavior_policies_1 = #TODO #Here we define behavioral policies learned on different subsets  ex: np.load(f'{name_dir}/behavior_policies_lightgbm_25.npy')

In [ ]:
tv_random = []
tv_pop = []
tv_frac = []
tv_pair = []
for i in range(num_polices):
    tv_random.append((np.abs(behavior_policies_1[i] - random_dist_test)).sum()/600)
    tv_pop.append((np.abs(behavior_policies_1[i] - pop_dist_test)).sum()/600)
    tv_frac.append((np.abs(behavior_policies_1[i] - frac_dist_test)).sum()/600)
    for j in range(i, num_polices):
        tv_pair.append((np.abs(behavior_policies_1[i] - behavior_policies_1[j])).sum()/600)
print(f'TV random={np.mean(tv_random)}')
print(f'TV popular={np.mean(tv_pop)}')
print(f'TV frequency={np.mean(tv_frac)}')
print(f'TV pairwise={np.mean(tv_pair)}')

### Compute offpolicy values

In [ ]:
def run_exp(all_action_dist, DM_method):
    metrics = ['ipw', 'snipw', 'sndr', 'dm']
    pi_e = all_action_dist[np.arange(bandit_feedback_test['action'].shape[0]), bandit_feedback_test['action'], 0].copy()
    cnt_inner_bootstraps = 100
    _alpha = 0.05
    CIs = []
    for n_size in [10000, 20000, 50000, 70000, 94124]:
        res = {}
        pi_e_subset = pi_e[:n_size].copy()
        r_hat_subset = r_hat[:n_size].copy()
        r_subset = bandit_feedback_test['reward'][:n_size].copy()
        dm_subset = DM_method[:n_size].copy()
        stats = {}
        print(n_size)
        for metric in metrics:
            stats[metric] = []
        behavior_policies =  #TODO #Here we define behavioral policies learned on different subsets  ex: np.load(f'{name_dir}/behavior_policies_lightgbm_25.npy')
        for i in range(num_polices):
            pi_0_subset= behavior_policies[i][:n_size]
            ipw_est_round_rewards = ipw(pi_e_subset, pi_0_subset, r_subset, r_hat_subset)
            snipw_est_round_rewards = snipw(pi_e_subset, pi_0_subset, r_subset, r_hat_subset)
            sndr_est_round_rewards = sndr(pi_e_subset, pi_0_subset, r_subset, r_hat_subset, dm_subset)
            dm_est_round_rewards = dm(dm_subset)

            for j in range(cnt_inner_bootstraps):
                bootstrap_idxs = np.random.randint(n_size, size=n_size)
                stats['ipw'].append(ipw_est_round_rewards[bootstrap_idxs].mean())
                stats['snipw'].append(snipw_est_round_rewards[bootstrap_idxs].mean())
                stats['sndr'].append(sndr_est_round_rewards[bootstrap_idxs].mean())
                stats['dm'].append(dm_est_round_rewards[bootstrap_idxs].mean())
        for metric in metrics:
            dct = {}
            values = stats[metric]
            dct['mean'] = np.mean(values)
            dct['95.0% CI (lower)'] = np.percentile(values, 100 * (_alpha / 2))
            dct['95.0% CI (upper)'] = np.percentile(values, 100 * (1.0 - _alpha / 2))
            res[metric] = dct
        CIs.append(res)
    return CIs

In [ ]:
CIs_1 = run_exp(all_action_dist_1_top_10, DM_method_1_top_10)

In [ ]:
CIs_2 = run_exp(all_action_dist_2_top_10, DM_method_2_top_10)

In [ ]:
CIs_3 = run_exp(all_action_dist_3_top_10, DM_method_3_top_10)

In [ ]:
import matplotlib.pyplot as plt
def plot_CIs(CIs_1, CIs_2, CIs_3, model_1, model_2, model_3):
    fig, ax = plt.subplots(4, figsize=(10, 10))
    
    x = [10000, 20000, 50000, 70000, 94124]

    colors = ['b', 'r', 'g', 'y']
    i = 0 
    for name in ['ipw', 'dm', 'sndr', 'snipw']:
        y_est = [estimated_ci[name]["mean"] for estimated_ci in CIs_1]
        y_up = [estimated_ci[name]["95.0% CI (upper)"] for estimated_ci in CIs_1]
        y_low = [estimated_ci[name]["95.0% CI (lower)"] for estimated_ci in CIs_1]
        
        ax[i].plot(x, y_est, '-', label=model_1, color = colors[0])
        ax[i].fill_between(x, y_low, y_up, alpha=0.2, color = colors[0])
        
        y_est = [estimated_ci[name]["mean"] for estimated_ci in CIs_2]
        y_up = [estimated_ci[name]["95.0% CI (upper)"] for estimated_ci in CIs_2]
        y_low = [estimated_ci[name]["95.0% CI (lower)"] for estimated_ci in CIs_2]
        
        ax[i].plot(x, y_est, '-', label=model_2, color = colors[1])
        ax[i].fill_between(x, y_low, y_up, alpha=0.2, color = colors[1])
        
        y_est = [estimated_ci[name]["mean"] for estimated_ci in CIs_3]
        y_up = [estimated_ci[name]["95.0% CI (upper)"] for estimated_ci in CIs_3]
        y_low = [estimated_ci[name]["95.0% CI (lower)"] for estimated_ci in CIs_3]
        
        ax[i].plot(x, y_est, '-', label=model_3, color = colors[2])
        ax[i].fill_between(x, y_low, y_up, alpha=0.2, color = colors[2])
        
        ax[i].set_title(name)
        i+=1

    fig.suptitle("OPE for " + " " + model_1 + " " + model_2 + " " + model_3, fontsize=16)
    fig.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
    plt.tight_layout()

In [ ]:
plot_CIs(CIs_1,CIs_2, CIs_3, 'UCB', 'Lin_UCB', 'Random')

# Using OBD library 

### To use the obd library, we must define bandit_feedback_test['pscore'].

In [ ]:
bandit_feedback_test['pscore'] = #TODO #Here we define behavioral policie learned on ONE subset

In [ ]:
import time
def run_exp_obd(learner_action_dist, lambda_, beta_1, beta_2):
    Vs = []
    CIs = []
    subsets = [10000, 20000, 50000, 70000, 94124]
        
    for n_size in subsets:
        start = time.time()
        bandit_feedback_subset = bandit_subset([0, n_size], bandit_feedback_test) #The first parameter is a slice of subset [a, b]

        ope = OffPolicyEvaluation(
            bandit_feedback=bandit_feedback_subset,
            ope_estimators=[InverseProbabilityWeighting(), DirectMethod(), SelfNormalizedDoublyRobust(),
                            SelfNormalizedInverseProbabilityWeighting(),
                            Exp_Smooth_IPS_Max(beta = beta_1), Exp_Smooth_IPS_Min(beta = beta_2), InverseProbabilityWeighting(lambda_ = lambda_, estimator_name='cips')]
        )

        estimated_rewards_by_reg_model_subset = estimated_rewards_by_reg_model[0: n_size, :]
        
        action_dist = learner_action_dist[:n_size, :]
        estimated_policy_value = ope.estimate_policy_values(
            action_dist=action_dist,
            estimated_rewards_by_reg_model=estimated_rewards_by_reg_model_subset,
        )

        estimated_ci = ope.estimate_intervals(
            action_dist=action_dist,
            estimated_rewards_by_reg_model=estimated_rewards_by_reg_model_subset,
            n_bootstrap_samples=100,
            random_state=12345,)
        end = time.time()
        print("n_size =", n_size, "time: ", end-start)
        Vs.append(estimated_policy_value)
        CIs.append(estimated_ci)
    return(Vs, CIs)

### Optimize parametrs of offpolicy methods

In [ ]:
def optimize_ope_parameters(learner_action_dist, lambda_s, beta_1s, beta_2s, k=10000):
    CIPS_MSE = []
    for lambda_ in lambda_s:
        ope = InverseProbabilityWeighting(lambda_=lambda_, estimator_name='cips')
        CIPS_MSE.append(ope._estimate_mse_score(
            reward=bandit_feedback_test['reward'][:k],
            action=bandit_feedback_test['action'][:k],
            pscore=bandit_feedback_test['pscore'][:k],
            action_dist=learner_action_dist[:k],
            position=bandit_feedback_test['position'][:k],
            use_bias_upper_bound = False))
        best_idx = np.argmin(np.array(CIPS_MSE))
        best_lambda_ = lambda_s[best_idx]
    
    ESIPSMAX_MSE = []
    for beta_1 in beta_1s:
        ope = Exp_Smooth_IPS_Max(beta = beta_1)
        ESIPSMAX_MSE.append(ope._estimate_mse_score(
            reward=bandit_feedback_test['reward'][:k],
            action=bandit_feedback_test['action'][:k],
            pscore=bandit_feedback_test['pscore'][:k],
            action_dist=learner_action_dist[:k],
            position=bandit_feedback_test['position'][:k],
            use_bias_upper_bound = False))
        best_idx = np.argmin(np.array(ESIPSMAX_MSE))
        best_beta_1 = beta_1s[best_idx]
        
    ESIPSMIN_MSE = []
    for beta_2 in beta_2s:
        ope = Exp_Smooth_IPS_Min(beta = beta_2)
        ESIPSMIN_MSE.append(ope._estimate_mse_score(
            reward=bandit_feedback_test['reward'][:k],
            action=bandit_feedback_test['action'][:k],
            pscore=bandit_feedback_test['pscore'][:k],
            action_dist=learner_action_dist[:k],
            position=bandit_feedback_test['position'][:k],
            use_bias_upper_bound = False))
        best_idx = np.argmin(np.array(ESIPSMIN_MSE))
        best_beta_2 = beta_2s[best_idx]
        
    return {'lambda_':best_lambda_, 'beta_1': best_beta_1, 'beta_2': best_beta_2}

In [ ]:
dict_opt_algo = {    
    'UCB': {'lambda_s': [1, 1.2, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2, 5, 6, 7, 8, 9, 9.5, 9.6, 9.7, 9.8, 9.9, 50, 55, 60, 65, 70, 71, 75, 79, np.inf], 
                 'beta_1s': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.98, 0.99, 1], 
                 'beta_2s': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.98, 0.99, 1]},
    
    'LinUCB': {'lambda_s': [1, 1.2, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2, 5, 6, 7, 8, 9, 9.5, 9.6, 9.7, 9.8, 9.9, 50, 55, 60, 65, 70, 71, 75, 79, np.inf], 
                 'beta_1s': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.98, 0.99, 1], 
                 'beta_2s': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.98, 0.99, 1]},
    
    'Random': {'lambda_s': [1, 1.2, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2, 5, 6, 7, 8, 9, 9.5, 9.6, 9.7, 9.8, 9.9, 50, 55, 60, 65, 70, 71, 75, 79, np.inf], 
                 'beta_1s': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.98, 0.99, 1], 
                 'beta_2s': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.98, 0.99, 1]},
}

### Run experiments

In [ ]:
learner_action_dist_1 = all_action_dist_1_top_10
learner_action_dist_2 = all_action_dist_2_top_10
learner_action_dist_3 = all_action_dist_3_top_10

In [ ]:
opt_params_ips_1 = optimize_ope_parameters(learner_action_dist_1, dict_opt_algo['UCB']['lambda_s'], dict_opt_algo['UCB']['beta_1s'], dict_opt_algo['UCB']['beta_2s'])
print(opt_params_ips_1)
Vs_1, CIs_1_obd = run_exp_obd(learner_action_dist_1, opt_params_ips_1['lambda_'], opt_params_ips_1['beta_1'], opt_params_ips_1['beta_2'])

In [ ]:
opt_params_ips_2 = optimize_ope_parameters(learner_action_dist_2, dict_opt_algo['LinUCB']['lambda_s'], dict_opt_algo['LinUCB']['beta_1s'], dict_opt_algo['LinUCB']['beta_2s'])
print(opt_params_ips_2)
Vs_2, CIs_2_obd = run_exp_obd(learner_action_dist_2, opt_params_ips_2['lambda_'], opt_params_ips_2['beta_1'], opt_params_ips_2['beta_2'])

In [ ]:
opt_params_ips_3 = optimize_ope_parameters(learner_action_dist_3, dict_opt_algo['Random']['lambda_s'], dict_opt_algo['Random']['beta_1s'], dict_opt_algo['Random']['beta_2s'])
print(opt_params_ips_3)
Vs_3, CIs_3_obd = run_exp_obd(learner_action_dist_3, opt_params_ips_3['lambda_'], opt_params_ips_3['beta_1'], opt_params_ips_3['beta_2'])

In [ ]:
import matplotlib.pyplot as plt
def plot_CIs_obd(CIs_1, CIs_2, CIs_3, model_1, model_2, model_3):
    fig, ax = plt.subplots(7, figsize=(10, 20))
    
    x = [10000, 20000, 50000, 70000, 94124]

    colors = ['b', 'r', 'g', 'y',  'aqua', 'pink', 'orange']
    i = 0 
    for name in ['ipw', 'dm', 'sndr', 'snipw', 'ESIPSMAX',  'ESIPSMIN', 'cips']:
        y_est = [estimated_ci[name]["mean"] for estimated_ci in CIs_1]
        y_up = [estimated_ci[name]["95.0% CI (upper)"] for estimated_ci in CIs_1]
        y_low = [estimated_ci[name]["95.0% CI (lower)"] for estimated_ci in CIs_1]
        
        ax[i].plot(x, y_est, '-', label=model_1, color = colors[0])
        ax[i].fill_between(x, y_low, y_up, alpha=0.2, color = colors[0])
        
        y_est = [estimated_ci[name]["mean"] for estimated_ci in CIs_2]
        y_up = [estimated_ci[name]["95.0% CI (upper)"] for estimated_ci in CIs_2]
        y_low = [estimated_ci[name]["95.0% CI (lower)"] for estimated_ci in CIs_2]
        
        ax[i].plot(x, y_est, '-', label=model_2, color = colors[1])
        ax[i].fill_between(x, y_low, y_up, alpha=0.2, color = colors[1])
        
        y_est = [estimated_ci[name]["mean"] for estimated_ci in CIs_3]
        y_up = [estimated_ci[name]["95.0% CI (upper)"] for estimated_ci in CIs_3]
        y_low = [estimated_ci[name]["95.0% CI (lower)"] for estimated_ci in CIs_3]
        
        ax[i].plot(x, y_est, '-', label=model_3, color = colors[2])
        ax[i].fill_between(x, y_low, y_up, alpha=0.2, color = colors[2])
        
        ax[i].set_title(name)
        i+=1

    fig.suptitle("OPE for " + " " + model_1 + " " + model_2 + " " + model_3, fontsize=16)
    fig.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
    plt.tight_layout()

In [ ]:
plot_CIs_obd(CIs_1_obd, CIs_2_obd, CIs_3_obd, 'UCB', 'LinUCB', 'Random')